In [ ]:
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
import os
import numpy as np
import pandas as pd
import datasets
import evaluate
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

In [ ]:
annotated_texts_df = pd.read_csv("annotated_texts_repr.csv", sep=",", encoding="utf-8")
annotated_texts_df['pop_sum'] = annotated_texts_df['pop_sum'].apply(lambda x: 0 if x < 2 else 1)
annotated_texts_df = annotated_texts_df.rename(columns={"pop_sum":"label"})

In [ ]:
df_pol_train, df_pol_test = train_test_split(annotated_texts_df, test_size=0.25, stratify=annotated_texts_df['label'], random_state=42)
df_pol_train = df_pol_train[["id", "text", "label"]]
df_pol_test = df_pol_test[["id", "text", "label"]]


In [ ]:
# Trasformo i dataset nel formato di huggingface
train = datasets.Dataset.from_pandas(pd.DataFrame(data=df_pol_train))
test = datasets.Dataset.from_pandas(pd.DataFrame(data=df_pol_test))
train = train.class_encode_column('label')
test = test.class_encode_column('label')

# ricavo il validation set
train_val = train.train_test_split(test_size=0.2, seed=8, stratify_by_column='label')
train = train_val["train"]
val = train_val["test"]

Stringifying the column:   0%|          | 0/8130 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/8130 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/2710 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/2710 [00:00<?, ? examples/s]

In [ ]:
# nome modello
model_name = 'dbmdz/bert-base-italian-xxl-cased'

# carico il modello
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# tokenizzatore associato
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-italian-xxl-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=512)

# tokenizzo i dataset
train = train.map(tokenize, batched=True)
val = val.map(tokenize, batched=True)
test = test.map(tokenize, batched=True)

train.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/6504 [00:00<?, ? examples/s]

Map:   0%|          | 0/1626 [00:00<?, ? examples/s]

Map:   0%|          | 0/2710 [00:00<?, ? examples/s]

In [ ]:
num_epochs = 1

training_args = TrainingArguments(
    f"{model_name}-ACTI_finetuned",
    eval_strategy = "epoch",
    logging_strategy="epoch",
    save_strategy = "epoch",
    logging_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=num_epochs,
    weight_decay=0,
    load_best_model_at_end=True,
)

In [ ]:
def acc_metric(eval_pred):
  accuracy_metric = evaluate.load("accuracy")
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)

  return accuracy_metric.compute(predictions=predictions, references=labels)

In [ ]:
# fine-tuning del modello
trainer = Trainer(
    model,
    training_args,
    train_dataset=train,
    eval_dataset=val,
    tokenizer=tokenizer,
    compute_metrics=acc_metric
)

trainer.train()

<ipython-input-11-5d724c2c7b30>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: f-melasi (f-melasi-universit-di-pisa) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.472800,0.418602,0.815498


TrainOutput(global_step=813, training_loss=0.47280444606203875, metrics={'train_runtime': 726.3308, 'train_samples_per_second': 8.955, 'train_steps_per_second': 1.119, 'total_flos': 1711274304061440.0, 'train_loss': 0.47280444606203875, 'epoch': 1.0})

In [ ]:
# predizioni sul validation set
predictions = trainer.predict(val)

# classification report sul validation set
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids
print(classification_report(true_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.85      0.83      0.84       955
           1       0.76      0.80      0.78       671

    accuracy                           0.82      1626
   macro avg       0.81      0.81      0.81      1626
weighted avg       0.82      0.82      0.82      1626



In [ ]:
# salvo il modello
trainer.save_model("FINETUNED_MODEL")

In [ ]:
log_history = trainer.state.log_history
import json
with open('log.json', 'w') as f:
    json.dump(log_history, f)

In [ ]:
predictions = trainer.predict(test)

predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids
print(classification_report(true_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.84      0.84      0.84      1592
           1       0.77      0.77      0.77      1118

    accuracy                           0.81      2710
   macro avg       0.81      0.80      0.81      2710
weighted avg       0.81      0.81      0.81      2710

